<a href="https://colab.research.google.com/github/sharvarijiwtode/PRODIGY_GA/blob/main/PRODIGY_GA_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>